In [149]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames, Plots, Statistics

In [150]:
## Function to Load Data.
function load_data(fname)
    data = DataFrame(CSV.File(fname, header=false)) |> DataFrame;
    return data
end;

In [151]:
hospital_mapping = CSV.read("../data/new_formulation/hospital_mapping.csv", DataFrame, pool=true);
distance_cost_vehicle_type_mapping = CSV.read("../data/new_formulation/distance_cost_vehicle_type_mapping.csv", DataFrame, pool=true);
fixed_cost_vehicle_type_mapping = CSV.read("../data/new_formulation/fixed_cost_vehicle_type_mapping.csv", DataFrame, pool=true);
vehicle_depot_mapping = CSV.read("../data/new_formulation/vehicle_depot_mapping.csv", DataFrame, pool=true);
distance_mapping = CSV.read("../data/new_formulation/distance_mapping.csv", DataFrame, pool=true);
vehicle_type = CSV.read("../data/new_formulation/vehicle_type.csv", DataFrame, pool=true);

In [152]:
patient_hospital_mapping = []
max_transfers = maximum(Matrix(hospital_mapping))
for i=1:max_transfers
    transfers = sort(Tuple.(findall(>=(i), Matrix(hospital_mapping))))
    for item in transfers
        push!(patient_hospital_mapping, item)
    end
#    push!(patient_hospital_mapping, sort(Tuple.(findall(>=(i), Matrix(hospital_mapping))))[0])
end

In [153]:
df_patient_hospital_mapping = DataFrame(patient_hospital_mapping)

Row,1,2
,Int64,Int64
1,1,1
2,2,1
3,2,18
4,3,1
5,3,18
6,4,1
7,5,1
8,6,1
9,6,9


In [154]:
CSV.write("../data/new_formulation/df_patient_hospital_mapping.csv", df_patient_hospital_mapping)

"../data/new_formulation/df_patient_hospital_mapping.csv"

In [155]:
distance_cost_vehicle_type_mapping = distance_cost_vehicle_type_mapping[:,2:end]

Row,cost_per_mile
,Int64
1,10
2,2


In [156]:
fixed_cost_vehicle_type_mapping = fixed_cost_vehicle_type_mapping[:,2:end]

Row,fixed_cost
,Int64
1,100
2,200


In [157]:
vehicle_depot_mapping = vehicle_depot_mapping[:,2:end]

Row,depot_num
,Int64
1,1
2,1
3,1
4,1
5,2


In [158]:
distance_mapping = distance_mapping[:,2:end]

Row,1,2,3,4,5,6,7,8,9,10
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,100,20,20,40,60,60,30,20,40
2,100,0,50,50,20,10,10,30,50,20
3,20,50,0,0,20,40,40,40,0,20
4,20,50,0,0,20,40,40,40,0,20
5,40,20,20,20,0,20,20,50,20,0
6,60,10,40,40,20,0,0,10,40,20
7,60,100,40,40,20,0,0,10,40,20
8,30,30,40,40,50,10,10,0,40,50
9,20,50,0,0,20,40,40,40,0,20


In [159]:
vehicle_type = vehicle_type[:,2:end]

Row,type
,Int64
1,1
2,1
3,2
4,1
5,2


In [160]:
(vehicle_depot_mapping[1,1] == 1)*2

2

### Defining Parameters

In [161]:
n_depots = 2
n_pickup = 4
n_destination = 4
V = n_depots + n_pickup + n_destination    # Total locations

depots = [1: 1: n_depots;]
pickups = [1+n_depots: 1: n_depots+n_pickup;]
destinations = [1+n_depots+n_pickup: 1: n_depots+n_pickup+n_destination;];

In [162]:
depots

2-element Vector{Int64}:
 1
 2

In [163]:
pickups

4-element Vector{Int64}:
 3
 4
 5
 6

In [164]:
destinations

4-element Vector{Int64}:
  7
  8
  9
 10

In [165]:
# n_vehicles_types = maximum(vehicle_type[!,1])
K = size(vehicle_type)[1] # n_vehicles

5

In [310]:
max_pickups_per_vehicle = 4

4

In [311]:
# Build model.
model = Model(Gurobi.Optimizer)
# set_optimizer_attribute(model, "OutputFlag", solver_output)


# Insert variable.
# We create a huge number of variables because we map multiple locations with the same index.
@variable(model, x[i=1:V, j=1:V, k=1:K], Bin)
@variable(model, y[k=1:K], Bin)
@variable(model, u[i=1:V, k=1:K], Int)


# Insert constraints.

## Related with x.
@constraint(model, [i=pickups[1]:pickups[n_pickup]], sum(sum(x[i,j,k] for j=1:V) for k=1:K) == 1)
@constraint(model, [i=pickups[1]:pickups[n_pickup]], sum(x[i,i+n_pickup,k] for k=1:K) == 1)
@constraint(model, [j=1:V, k=1:K], sum(x[i,j,k] for i=1:V) - sum(x[j,i,k] for i=1:V) == 0)
@constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=1:V) for i=pickups[1]:pickups[n_pickup]) <= max_pickups_per_vehicle)

## Related with y.
@constraint(model, [k=1:K], sum(x[depots[vehicle_depot_mapping[k,1]],j,k] for j=pickups[1]:pickups[n_pickup]) == y[k])
@constraint(model, [i=1:V, j=1:V, k=1:K], x[i,j,k] <= y[k])

## Related with u.
@constraint(model, [i=depots[1]:depots[n_depots], k=1:K], u[i,k] == 1)
@constraint(model, [i=n_depots+1:V, k=1:K], 2 <= u[i,k] <= V)
@constraint(model, [i=n_depots+1:V, j=n_depots+1:V, k=1:K], u[i,k] - u[j,k] + 1 <= (V-1)*(1-x[i,j,k]))

# Objective.
@objective(model, Min, 
    sum(fixed_cost_vehicle_type_mapping[vehicle_type[k,1],1]*y[k] for k=1:K) +
    sum(sum(sum(distance_cost_vehicle_type_mapping[vehicle_type[k,1],1]*distance_mapping[i,j]*x[i,j,k] for i=1:V) for j=1:V) for k=1:K)
)


# Optimize.
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 938 rows, 595 columns and 3315 nonzeros
Model fingerprint: 0xf4dda49e
Variable types: 40 continuous, 555 integer (505 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+01, 1e+03]
  Bounds range     [2e+00, 1e+01]
  RHS range        [1e+00, 8e+00]
Presolve removed 569 rows and 280 columns
Presolve time: 0.00s
Presolved: 369 rows, 315 columns, 1345 nonzeros
Variable types: 0 continuous, 315 integer (275 binary)
Found heuristic solution: objective 900.0000000

Root relaxation: objective 5.400000e+02, 47 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [312]:
value.(x)

10×10×5 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [313]:
x_sol = Dict()
u_sol = Dict()
path_mappings = Dict()
sequences = Dict()

Dict{Any, Any}()

In [314]:
for i=1:K
    x_sol[i] = DataFrame(value.(x)[:,:,i], :auto);
    u_sol[i] = value.(u)[:,i];
    path_mappings[i] = sort(Tuple.(findall(>=(1), Matrix(x_sol[i]))))
    sequences[i] = sort(DataFrame(Start=[1:1:size(u_sol[i])[1];], Sequence=u_sol[i]), :Sequence)
end

In [315]:
sequence_travelled = Dict()

for vehicle_num = 1:5
    if size(path_mappings[vehicle_num])[1] == 0
        continue
    else
        path = path_mappings[vehicle_num]
        sequence = sort(sequences[vehicle_num], :Start)
        sequence_dict = Dict(pairs(sequence.Sequence))
#         print(sequence_dict)
        start_nodes = path[:,1]
        nodes = []
        sequence = []
        
        for node_index = 1:size(start_nodes)[1]
            node_num = start_nodes[node_index][1]
            seq = sequence_dict[start_nodes[node_index][1]]
            push!(nodes, node_num)
            push!(sequence, seq)
#             println(nodes)
#             println(sequence_dict[start_nodes[node_index][1]])
        end
        
        sequence_travelled[vehicle_num] = sort(DataFrame(Nodes=nodes, VistedAt=sequence), :VistedAt)[:,1]
        
#         nodes_travelled[vehicle_num] = nodes
#         sequence_travelled[vehicle_num] = sequence
    end
end

In [316]:
sequence_travelled

Dict{Any, Any} with 1 entry:
  3 => Any[1, 3, 7, 6, 10, 5, 9, 4, 8]

In [318]:
total_cost = objective_value.(model)

540.0

I just tried constrained the number of pickups to be constrained by 2 patients (since we only had 4 right now), and this is what we get! It choses 2 vehicles to do different patients!! 3-4 done by 5th vehicle and 5,6 done by 3rd vehicle and both are vehicles of type 2 (electric) so the choice of vehicles also works good

I also tried putting both the vehicles in different depots and it works. Just that they are currently going back to their original depot. Maybe we can say that it goes back to original, otherwise there would have been a complexity of capacity and vacancy at each depot, and thus to not do that, we put this constrain.

In [144]:
# CSV.write("../data/new_formulation/df_sequence_mapping.csv", df_sequence_mapping)

"../data/new_formulation/df_sequence_mapping.csv"

In [145]:
# CSV.write("../data/new_formulation/result.csv", a)

"../data/new_formulation/result.csv"